# Machine Learning - Final project
### Empathy Prediction using the [Young people survey](https://www.kaggle.com/miroslavsabo/young-people-survey/) dataset

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":24,"axes.titlesize":24,"axes.labelsize":24})  
pd.options.display.max_columns = 150
pd.options.display.max_rows = 150

#### Loading datasets

In [89]:
df = pd.read_csv('./input/responses.csv')
columns_desc = pd.read_csv('./input/columns.csv')

In [90]:
df.shape

(1010, 150)

#### Defining way to access description easily

In [70]:
all_columns =list(df)

desc = {}
for col_name in all_columns:
    desc[col_name] = columns_desc.loc[columns_desc['short'] == col_name].iloc[0][0]
    
def print_desc(column_name):
    print(column_name+" --> "+desc[column_name])

Example:

In [71]:
for col in all_columns[:5]:
    print_desc(col)

Music --> I enjoy listening to music.
Slow songs or fast songs --> I prefer.
Dance --> Dance, Disco, Funk
Folk --> Folk music
Country --> Country


## Data Exploration and preprocessing

In [143]:
df.describe()

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,Punk,"Hiphop, Rap","Reggae, Ska","Swing, Jazz",Rock n roll,Alternative,Latino,"Techno, Trance",Opera,Movies,Horror,Thriller,Comedy,Romantic,Sci-fi,War,Fantasy/Fairy tales,Animated,Documentary,Western,Action,History,Psychology,Politics,Mathematics,Physics,Internet,PC,Economy Management,Biology,Chemistry,Reading,Geography,Foreign languages,Medicine,Law,Cars,Art exhibitions,Religion,"Countryside, outdoors",Dancing,Musical instruments,Writing,Passive sport,Active sport,Gardening,Celebrities,Shopping,Science and technology,Theatre,Fun with friends,Adrenaline sports,Pets,Flying,Storm,Darkness,Heights,Spiders,Snakes,Rats,Ageing,Dangerous dogs,Fear of public speaking,Healthy eating,Daily events,Prioritising workload,Writing notes,Workaholism,Thinking ahead,Final judgement,Reliability,Keeping promises,Loss of interest,Friends versus money,Funniness,Fake,Criminal damage,Decision making,Elections,Self-criticism,Judgment calls,Hypochondria,Empathy,Eating to survive,Giving,Compassion to animals,Borrowed stuff,Loneliness,Cheating in school,Health,Changing the past,God,Dreams,Charity,Number of friends,Waiting,New environment,Mood swings,Appearence and gestures,Socializing,Achievements,Responding to a serious letter,Children,Assertiveness,Getting angry,Knowing the right people,Public speaking,Unpopularity,Life struggles,Happiness in life,Energy levels,Small - big dogs,Personality,Finding lost valuables,Getting up,Interests or hobbies,Parents' advice,Questionnaires or polls,Finances,Shopping centres,Branded clothing,Entertainment spending,Spending on looks,Spending on gadgets,Spending on healthy eating,Age,Height,Weight,Number of siblings
count,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.00000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.00000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000
mean,4.736318,3.330348,3.112438,2.288557,2.123383,2.956219,2.763184,3.470647,3.773134,2.358209,2.446766,2.915423,2.770149,2.757214,3.140299,2.830846,2.834826,2.333333,2.141294,4.615920,2.793035,3.384080,4.497512,3.488557,3.112438,3.153234,3.751244,3.790050,3.647761,2.121393,3.539303,3.203980,3.139303,2.591045,2.332338,2.062687,4.18408,3.137313,2.639801,2.663682,2.153234,3.166169,3.081592,3.783085,2.509453,2.255721,2.678607,2.581095,2.272637,3.696517,2.458706,2.320398,1.891542,3.413930,3.297512,1.905473,2.362189,3.275622,3.235821,3.025871,4.558209,2.950249,3.343284,2.058

In [161]:
df.rename(columns={'Left - right handed': 'LeftRightHanded', 'Village - town': 'VillageTown', 'Internet usage': 'InternetUsage', 'Only child': 'OnlyChild', 'House - block of flats': 'HouseFlats'}, inplace=True)

I notice that some attributes like Music have really skewed distributions, later on I could try to apply some log transformations and compare the performances

### Missing values

Let's start by looking at how many rows are missing a class label, I will delete these rows since it makes no sense to impute them, train or test using them

In [86]:
df['Empathy'].isna().sum()

5

In [91]:
df.dropna(subset=['Empathy'], inplace=True)

In [92]:
df.shape

(1005, 150)

Let's now look at the percentage of missing values in all columns in desc order

In [94]:
df_na = (df.isnull().sum() / len(df)) * 100
df_na = df_na.drop(df_na[df_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio': df_na})
missing_data.head(150).T

,Height,Weight,Passive sport,Chemistry,Geography,Theatre,Documentary,Smoking,Latino,Punk,Criminal damage,Compassion to animals,Final judgement,"Reggae, Ska",Gardening,Alternative,Rock n roll,"Techno, Trance",Age,Classical music,"Swing, Jazz",Movies,Biology,Reading,Giving,PC,Gender,Number of siblings,Responding to a serious letter,Daily events,Science and technology,Art exhibitions,Friends versus money,Writing,"Countryside, outdoors",Rock,Folk,Self-criticism,Spiders,Country,Economy Management,Energy levels,Workaholism,Psychology,Foreign languages,Prioritising workload,Getting up,Medicine,Alcohol,Socializing,Village - town,Pets,Active sport,Reliability,Loss of interest,Fun with friends,Cars,House - block of flats,Funniness,Decision making,Dance,"Hiphop, Rap",Questionnaires or polls,Finding lost valuables,Personality,Small - big dogs,Happiness in life,Getting angry,Children,Mood swings,Western,Internet,Cheating in school,Hypochondria,Judgment calls,Romantic,Comedy,Fantasy/Fairy tales,Animated,Religion,Physics,Dancing,Metal or Hardrock,Pop,Mathematics,Music,Appearence and gestures,Spending on looks,Life struggles,Interests or hobbies,Charity,Finances,Elections,Thinking ahead,Writing notes,Healthy eating,Entertainment spending,Rats,Heights,Left - right handed
Missing Ratio,1.99005,1.99005,1.492537,0.995025,0.895522,0.79602,0.79602,0.79602,0.79602,0.79602,0.696517,0.696517,0.696517,0.696517,0.696517,0.696517,0.696517,0.696517,0.696517,0.696517,0.597015,0.597015,0.597015,0.597015,0.597015,0.597015,0.597015,0.597015,0.597015,0.597015,0.597015,0.597015,0.597015,0.597015,0.597015,0.597015,0.497512,0.497512,0.497512,0.497512,0.497512,0.497512,0.497512,0.497512,0.497512,0.497512,0.497512,0.497512,0.497512,0.497512,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.39801,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507,0.298507


The percentage of missing values is not a lot, we can easily impute them without introducing too much bias by using the mode

In [95]:
df[all_columns]=df[all_columns].fillna(df.mode().iloc[0])

In [97]:
df.isnull().any().any()

False

There are no missing values now

### Data types

In [17]:
df['Music'] = df['Music'].astype(int)

['a', 'b', 'c', 'd']

In [105]:
df.dtypes[:10]

Music                       float64
Slow songs or fast songs    float64
Dance                       float64
Folk                        float64
Country                     float64
Classical music             float64
Musical                     float64
Pop                         float64
Rock                        float64
Metal or Hardrock           float64
dtype: object

In [109]:
float_cols = df.dtypes == np.float64

In [115]:
fake_float = list(df.loc[:, float_cols])

In [116]:
len(fake_float)

134

134 columns are float but should be int since the decimal value is always 0, let's convert them

In [117]:
df[fake_float] = df[fake_float].astype(int)

### One hot encoding
Since the algorithms that I have in mind do not handle categorical values I will need to perform one hot encoding or encode values in numbers for ordinal data

Let's look at categorical columns and decide which ones can be ordinal

In [163]:
categorical = df.dtypes == np.object

In [164]:
df.loc[:, categorical].T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,...,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009
Smoking,never smoked,never smoked,tried smoking,former smoker,tried smoking,never smoked,tried smoking,current smoker,tried smoking,never smoked,current smoker,tried smoking,tried smoking,never smoked,current smoker,former smoker,current smoker,tried smoking,tried smoking,former smoker,never smoked,tried smoking,tried smoking,tried smoking,tried smoking,tried smoking,tried smoking,never smoked,former smoker,tried smoking,former smoker,tried smoking,current smoker,tried smoking,tried smoking,tried smoking,never smoked,former smoker,tried smoking,never smoked,never smoked,tried smoking,never smoked,tried smoking,current smoker,never smoked,current smoker,tried smoking,current smoker,tried smoking,tried smoking,never smoked,current smoker,tried smoking,current smoker,current smoker,current smoker,current smoker,never smoked,current smoker,tried smoking,current smoker,never smoked,tried smoking,former smoker,tried smoking,tried smoking,never smoked,tried smoking,tried smoking,former smoker,tried smoking,former smoker,former smoker,tried smoking,...,tried smoking,tried smoking,never smoked,former smoker,never smoked,tried smoking,current smoker,tried smoking,current smoker,never smoked,current smoker,tried smoking,tried smoking,tried smoking,tried smoking,never smoked,former smoker,never smoked,tried smoking,former smoker,current smoker,current smoker,never smoked,tried smoking,current smoker,never smoked,tried smoking,never smoked,current smoker,tried smoking,tried smoking,tried smoking,never smoked,tried smoking,former smoker,current smoker,tried smoking,tried smoking,current smoker,current smoker,current smoker,tried smoking,current smoker,tried smoking,never smoked,tried smoking,tried smoking,tried smoking,tried smoking,current smoker,former smoker,tried smoking,current smoker,tried smoking,former smoker,former smoker,tried smoking,never smoked,former smoker,current smoker,tried smoking,former smoker,tried smoking,tried smoking,never smoked,never smoked,former smoker,current smoker,former smoker,former smoker,current smoker,never smoked,tried smoking,tried smoking,tried smoking
Alcohol,drink a lot,drink a lot,drink a lot,drink a lot,social drinker,never,social drinker,drink a lot,social drinker,drink a lot,social drinker,never,social drinker,social drinker,social drinker,social drinker,drink a lot,social drinker,social drinker,drink a lot,never,social drinker,social drinker,social drinker,never,social drinker,social drinker,social drinker,social drinker,social drinker,drink a lot,social drinker,social drinker,social drinker,social drinker,social drinker,social drinker,social drinker,social drinker,never,social drinker,social drinker,never,drink a lot,social drinker,social drinker,social drinker,social drinker,social drinker,social drinker,social drinker,never,social drinker,social drinker,social drinker,drink a lot,drink a lot,drink a lot,social drinker,drink a lot,social drinker,social drinker,social drinker,social drinker,social drinker,social drinker,social drinker,never,drink a lot,social drinker,social drinker,never,social drinker,social drinker,social drinker,...,social drinker,social drinker,social drinker,social drinker,never,never,social drinker,drink a lot,social drinker,social drinker,drink a lot,social drinker,social drinker,social drinker,social drinker,never,drink a lot,never,social drinker,social drinker,drink a lot,drink a lot,social drinker,social drinker,social drinker,social drinker,social

In [128]:
categorical_cols = list(df.loc[:, categorical])

In [215]:
for col in categorical_cols:
    print_desc(col)

Smoking --> Smoking habits
Alcohol --> Drinking
Punctuality --> Timekeeping.
Lying --> Do you lie to others?
Internet usage --> How much time do you spend online?
Gender --> Gender
Left - right handed --> I am
Education --> Highest education achieved
Only child --> I am the only child
Village - town --> I spent most of my childhood in a
House - block of flats --> I lived most of my childhood in a


There are a lot of ordinal columns, which is better because we will not occur in a feature explosion using OHE, I think the ordinal values could be:

* Punctuality
* InternetUsage
* Education

Some other that only have True/False or Yes/No or in general 2 possible values can be encoded using the same attribute and an integer value (0 or 1), these features are:

* Gender
* LeftRightHanded
* OnlyChild
* VillageTown
* HouseFlats

The remaining categorical attributes are the ones that I think are the most important to predict empathy or that are not really ordinal and I would not want to introduce a bias by considering them as ordinal, these attributes are:

* Smoking
* Lying
* Alcohol

In [145]:
print("Unique values of Smoking")
print(df.Smoking.unique())

print("\nUnique values of Lying")
print(df.Lying.unique())


Unique values of Smoking
['never smoked' 'tried smoking' 'former smoker' 'current smoker']

Unique values of Lying
['never' 'sometimes' 'only to avoid hurting someone'
 'everytime it suits me']


In [153]:
df.shape

(1005, 158)

In [148]:
def OHE_Smoking_N(value):
    return 1 if value=='never smoked' else 0

def OHE_Smoking_T(value):
    return 1 if value=='tried smoking' else 0

def OHE_Smoking_F(value):
    return 1 if value=='former smoker' else 0

def OHE_Smoking_C(value):
    return 1 if value=='current smoker' else 0

In [149]:
df['Smoking_Never'] = df.Smoking.apply(OHE_Smoking_N)
df['Smoking_Tried'] = df.Smoking.apply(OHE_Smoking_T)
df['Smoking_Former'] = df.Smoking.apply(OHE_Smoking_F)
df['Smoking_Current'] = df.Smoking.apply(OHE_Smoking_C)

In [151]:
def OHE_Lying_N(value):
    return 1 if value=='never' else 0

def OHE_Lying_S(value):
    return 1 if value=='sometimes' else 0

def OHE_Lying_O(value):
    return 1 if value=='only to avoid hurting someone' else 0

def OHE_Lying_E(value):
    return 1 if value=='everytime it suits me' else 0

In [152]:
df['Lying_Never'] = df.Lying.apply(OHE_Lying_N)
df['Lying_Sometimes'] = df.Lying.apply(OHE_Lying_S)
df['Lying_Only'] = df.Lying.apply(OHE_Lying_O)
df['Lying_Everytime'] = df.Lying.apply(OHE_Lying_E)

In [188]:
def OHE_Alcohol_D(value):
    return 1 if value=='drink a lot' else 0

def OHE_Alcohol_S(value):
    return 1 if value=='social drinker' else 0

def OHE_Alcohol_N(value):
    return 1 if value=='never' else 0

In [189]:
df['Alcohol_Drink'] = df.Alcohol.apply(OHE_Alcohol_D)
df['Alcohol_Social'] = df.Alcohol.apply(OHE_Alcohol_S)
df['Alcohol_Never'] = df.Alcohol.apply(OHE_Alcohol_N)

#### True False OHE

In [171]:
def OHE_Male(value):
    return 1 if value=='male' else 0

In [172]:
df['Male'] = df.Gender.apply(OHE_Male)

In [166]:
def OHE_Right(value):
    return 1 if value=='right handed' else 0

In [167]:
df['Right_Handed'] = df.LeftRightHanded.apply(OHE_Right)

In [174]:
def OHE_Only(value):
    return 1 if value=='yes' else 0

In [175]:
df['Only_Child'] = df.OnlyChild.apply(OHE_Only)

In [182]:
def OHE_City(value):
    return 1 if value=='city' else 0

In [183]:
df['City'] = df.VillageTown.apply(OHE_City)

In [184]:
def OHE_House(value):
    return 1 if value=='house/bungalow' else 0

In [185]:
df['House'] = df.HouseFlats.apply(OHE_House)

#### Ordinal

In [192]:
print("Unique values of Punctuality")
print(df.Punctuality.unique())

Unique values of Punctuality
['i am always on time' 'i am often early' 'i am often running late']


In [194]:
def OHE_Punctuality_Ordinal(value):
    if value=='i am always on time':
        return 1
    if value=='i am often early':
        return 2 
    if value=='i am often running late':
        return 0
    #should not arrive here
    return 0

In [195]:
df['Punctual'] = df.Punctuality.apply(OHE_Punctuality_Ordinal)

In [197]:
print("Unique values of InternetUsage")
print(df.InternetUsage.unique())

Unique values of InternetUsage
['few hours a day' 'most of the day' 'less than an hour a day'
 'no time at all']


In [198]:
def OHE_InternetUsage_Ordinal(value):
    if value=='most of the day':
        return 3
    if value=='few hours a day':
        return 2 
    if value=='less than an hour a day':
        return 1
    if value=='no time at all':
        return 0
    #should not arrive here
    return 0

In [199]:
df['InternetUsage'] = df.InternetUsage.apply(OHE_InternetUsage_Ordinal)

In [202]:
print("Unique values of Education")
print(df.Education.unique())

Unique values of Education
['college/bachelor degree' 'secondary school' 'primary school'
 'masters degree' 'doctorate degree' 'currently a primary school pupil']


In [203]:
def OHE_Education_Ordinal(value):
    if value=='doctorate degree':
        return 5
    if value=='masters degree':
        return 4 
    if value=='college/bachelor degree':
        return 3
    if value=='secondary school':
        return 2
    if value=='primary school':
        return 1
    if value=='currently a primary school pupil':
        return 0
    #should not arrive here
    return 0

In [204]:
df['Education'] = df.Education.apply(OHE_Education_Ordinal)

#### Deleting categorical columns that were encoded

In [208]:
encoded_columns = ['Smoking','Lying','Alcohol','Gender','LeftRightHanded','OnlyChild','VillageTown','HouseFlats','Punctuality']

In [211]:
df = df.drop(encoded_columns, axis=1)

In [213]:
df.shape

(1005, 158)

In [216]:
pd.options.display.max_columns = 158
pd.options.display.max_rows = 158

In [218]:
df.head()

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,Punk,"Hiphop, Rap","Reggae, Ska","Swing, Jazz",Rock n roll,Alternative,Latino,"Techno, Trance",Opera,Movies,Horror,Thriller,Comedy,Romantic,Sci-fi,War,Fantasy/Fairy tales,Animated,Documentary,Western,Action,History,Psychology,Politics,Mathematics,Physics,Internet,PC,Economy Management,Biology,Chemistry,Reading,Geography,Foreign languages,Medicine,Law,Cars,Art exhibitions,Religion,"Countryside, outdoors",Dancing,Musical instruments,Writing,Passive sport,Active sport,Gardening,Celebrities,Shopping,Science and technology,Theatre,Fun with friends,Adrenaline sports,Pets,Flying,Storm,Darkness,Heights,Spiders,Snakes,Rats,Ageing,Dangerous dogs,Fear of public speaking,Healthy eating,Daily events,Prioritising workload,Writing notes,Workaholism,Thinking ahead,Final judgement,Reliability,Keeping promises,Loss of interest,Friends versus money,Funniness,Fake,Criminal damage,Decision making,Elections,Self-criticism,Judgment calls,Hypochondria,Empathy,Eating to survive,Giving,Compassion to animals,Borrowed stuff,Loneliness,Cheating in school,Health,Changing the past,God,Dreams,Charity,Number of friends,Waiting,New environment,Mood swings,Appearence and gestures,Socializing,Achievements,Responding to a serious letter,Children,Assertiveness,Getting angry,Knowing the right people,Public speaking,Unpopularity,Life struggles,Happiness in life,Energy levels,Small - big dogs,Personality,Finding lost valuables,Getting up,Interests or hobbies,Parents' advice,Questionnaires or polls,InternetUsage,Finances,Shopping centres,Branded clothing,Entertainment spending,Spending on looks,Spending on gadgets,Spending on healthy eating,Age,Height,Weight,Number of siblings,Education,Smoking_Never,Smoking_Tried,Smoking_Former,Smoking_Current,Lying_Never,Lying_Sometimes,Lying_Only,Lying_Everytime,Male,Right_Handed,Only_Child,City,House,Alcohol_Drink,Alcohol_Social,Alcohol_Never,Punctual
0,5,3,2,1,2,2,1,5,5,1,1,1,1,1,3,1,1,1,1,5,4,2,5,4,4,1,5,5,3,1,2,1,5,1,3,3,5,3,5,3,3,3,3,5,3,1,1,1,1,5,3,3,2,1,5,5,1,4,4,2,5,4,4,1,1,1,1,1,5,3,1,3,2,4,2,2,5,4,2,5,4,4,1,3,5,1,1,3,4,1,3,1,3,1,4,5,4,3,2,1,1,1,4,2,3,3,4,3,4,3,4,3,5,1,1,3,5,5,1,4,5,1,4,3,2,3,4,3,2,3,4,5,3,3,1,3,20,163,48,1,3,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1
1,4,4,2,1,1,1,2,3,5,4,4,1,3,1,4,4,2,1,1,5,2,2,4,3,4,1,3,5,4,1,4,1,3,4,5,2,4,4,5,1,1,4,4,5,1,2,2,2,1,1,1,1,1,1,1,1,2,3,3,2,4,2,5,1,1,1,2,1,1,1,3,1,4,3,3,2,4,5,4,1,4,4,3,4,3,2,1,2,5,4,4,1,2,1,2,4,3,2,4,4,4,1,3,1,3,3,4,4,4,4,2,4,2,2,5,4,4,4,1,4,3,5,3,4,5,3,2,3,2,3,4,1,4,2,5,2,19,163,58,2,3,1,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,2
2,5,5,2,2,3,4,5,3,5,3,4,1,4,3,5,5,5,1,3,5,3,4,4,2,4,2,5,5,2,2,1,1,2,1,5,2,4,2,4,1,1,5,2,5,2,3,1,5,5,5,5,5,5,5,2,1,1,4,2,5,5,5,5,1,1,1,1,1,1,1,1,1,2,3,1,2,5,3,5,3,4,5,1,5,2,4,1,3,5,4,4,1,5,5,5,4,2,5,3,2,5,5,1,3,3,2,3,4,3,5,3,4,4,3,4,3,2,4,4,4,4,3,3,3,4,5,3,1,2,2,4,1,4,3,4,2,20,176,67,2,2,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0
3,5,3,2,1,1,1,1,2,2,1,4,2,2,1,2,5,1,2,1,5,4,4,3,3,4,3,1,2,5,1,2,4,4,5,4,1,3,1,2,3,3,5,4,4,2,5,1,5,4,1,1,1,3,1,1,1,2,4,3,1,2,1,1,2,1,1,3,5,5,5,4,5,5,3,4,4,4,5,3,1,3,4,5,2,1,1,5,5,5,5,4,3,3,1,1,2,5,5,5,1,5,4,3,3,1,1,1,5,3,1,3,3,2,5,5,4,5,3,3,2,2,1,2,1,1,3,2,4,3,2,4,3,3,4,4,1,22,172,59,1,3,0,0,1,0,0,0,1,0,0,1,1,1,1,1,0,0,2
4,5,3,4,3,2,4,3,5,3,1,2,5,3,2,1,2,4,2,2,5,4,4,5,2,3,3,4,4,3,1,4,3,2,3,2,2,2,2,2,3,3,5,2,3,3,2,3,1,4,4,1,3,1,3,1,4,3,3,3,2,4,2,1,1,2,1,1,1,1,2,2,4,3,4,3,1,2,3,5,5,5,4,2,3,3,2,1,3,5,5,5,1,3,1,3,3,4,3,5,3,4,5,3,3,3,3,4,2,3,3,3,3,5,4,2,3,5,5,2,3,5,3,3,2,4,3,3,3,2,4,3,4,3,3,2,4,20,170,59,1,2,0,1,0,0,0,0,0,1,0,1,0,0,1,0,1,0,1
